## Coleta de dados

In [1]:
#!pip install spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
import sys
import csv

import pandas as pd

In [2]:
sp = spotipy.Spotify(client_credentials_manager= SpotifyClientCredentials(client_id="a74e8eedfb694efc8e7162e43ec0cb3f", client_secret="bc7e77c0222741b4b2e7bebb37da69bf"))

In [3]:
'''
 dado o id/url/urn de um artista, retorna o id 
 sp.artist(url/urn/id) retorna um objeto artista que contém as infos de um artista/banda
 nesse caso queremos apenas o id desse objeto
''' 
def get_artist_id(url):
    return sp.artist(url)['id']

In [4]:
# dado o id do artista, retorna as faixas mais populares
def show_artist_top_tracks(id):
  response = sp.artist_top_tracks(id)
  for track in response['tracks']:
    print(track['name'])

In [5]:
# dado o id do artista, retorna os artistas relacionados
def show_artist_related_artists(id):
    related_artists = sp.artist_related_artists(id)
    for artist in related_artists['artists']:
        print (artist['name'])

In [6]:
def get_artist_related_artists(id):
    related_artists = {'id':[], 'name': [], 'popularity': [], 'followers':[], 'position':[]}
    response = sp.artist_related_artists(id)
    pos = 1
    for artist in response['artists']:
        related_artists['id'].append(artist['id'])
        related_artists['name'].append(artist['name'])
        related_artists['popularity'].append(artist['popularity'])
        related_artists['followers'].append(artist['followers']['total'])
        related_artists['position'].append(pos)
        pos+=1
    r_a = pd.DataFrame(related_artists)
    return r_a
    

In [7]:
# dado o id do artista, retorna todos os álbuns que o artista fez ou colaborou
def get_artist_albums(artist_id):
    albums = []
    results = sp.artist_albums(artist_id, country = 'BR')
    albums.extend(results['items'])
    while results['next']:
        results = sp.next(results)
        albums.extend(results['items'])
    return albums

In [8]:
# dada uma lista de álbuns, retorna um dicionário id: name com os álbuns autorais do artista
def get_authorial_albums(albums):
    albums_id_name = {}
    for album in albums:
        if album['album_group'] != 'appears_on':
            albums_id_name[album['id']] = album['name']
    return albums_id_name

In [9]:
'''
 dado o id do álbum, retorna todas as músicas de um álbum de um artista 
 gera um dicionário com as seguintes informações das músicas do álbum: álbum, 
 número da música no álbum, id, nome, popularidade, duração, 
 data e artistas envolvidos
''' 
def get_album_tracks(album_id):
    spotify_album = {}
    album = sp.album(album_id)
    
    results = sp.album_tracks(album_id) # para cada álbum, retorna as informações das músicas
    tracks = results['items']
  
    for track in tracks:
        track_id = track['id']
        track_ = sp.track(track_id) # essa função tem mais info da faixa que acessando pelo álbum

        spotify_album[track_id] = {}
    
        spotify_album[track_id]['album'] = track_['album']['name']
        spotify_album[track_id]['album_type'] = album['album_type']
    
        spotify_album[track_id]['track_id'] = track_id
        spotify_album[track_id]['track_number'] = track_['track_number']
        
        spotify_album[track_id]['name'] = track_['name']
        spotify_album[track_id]['popularity'] = track_['popularity']
        spotify_album[track_id]['duration_ms'] = track_['duration_ms']
        spotify_album[track_id]['release_date'] = track_['album']['release_date']

        track_artists = track_['artists']
        
        spotify_album[track_id]['artists'] = []
        for artist in track_artists:
            spotify_album[track_id]['artists'].append(artist['name'])
        
    return spotify_album

In [10]:
'''
 dado um dicionario com ids e nomes de álbuns (provenientes de get_authorial_albums), 
 retorna todas as músicas de todos os albuns de um artista
''' 

def get_all_albums_tracks(authorial_albums):
    spotify_albums = []
    albums_names = []
  
    for album_id, album_name in authorial_albums.items():
        print('Adicionando: ',album_id,album_name)

        if album_name not in albums_names:
            albums_names.append(album_name)
            album_tracks = get_album_tracks(album_id) 
             
        for track_infos in album_tracks.values():
            spotify_albums.append(track_infos)
            
    print('Finalizado!')
    
    return spotify_albums

In [11]:
def convert_to_csv(filepath, name):
    df = pd.DataFrame(filepath)
    df.to_csv('../data/'+name+'.csv')
    return df.head()

In [12]:
calcinhapreta = "https://open.spotify.com/artist/6Kps94g1Npexh3LrinIOvC"
artist_id = get_artist_id(calcinhapreta)
   
if artist_id:
    print("Id da Calcinha Preta: %s", artist_id) 

    print("Os artistas relacionados a Calcinha Preta segundo o Spotify:")
    show_artist_related_artists(artist_id)  
else:
    logger.error("Can't find artist: %s", artist)

Id da Calcinha Preta: %s 6Kps94g1Npexh3LrinIOvC
Os artistas relacionados a Calcinha Preta segundo o Spotify:
Aviões do Forró
Cavaleiros do Forró
Banda Magníficos
Solange Almeida
Avine Vinny
Banda Calypso
Márcia Fellipe
Limão Com Mel
Companhia do Calypso
Luan Estilizado
Forró do Muido
Mastruz Com Leite
Banda A Loba
Malla 100 Alça
Bonde do Forró
Jonas Esticado
Chicabana
Asa De Águia
Desejo De Menina
Cavalo De Pau


In [13]:
related_artists = get_artist_related_artists(artist_id)
print(related_artists)

                        id                  name  popularity  followers  \
0   4mnhyGRAuExTL4J0soT4za       Aviões do Forró          62    1867607   
1   421vs0uTmJqkbaYBd5ww42   Cavaleiros do Forró          54     352264   
2   74B3SJ1xNHK3VS4T9i2TvQ      Banda Magníficos          52     310280   
3   3Hew3AuvrbKxCbehT4Rorq       Solange Almeida          57     946355   
4   4KDxYlnGZEiS60pAnfZf4W           Avine Vinny          62     436874   
5   2qXtddeMnGLeLuIfhnJY3P         Banda Calypso          58     170805   
6   3OxZ4asD88P1lJJmLcXz4K        Márcia Fellipe          65    1182439   
7   2gzwjs75n4HZZfgnVGxoqI         Limão Com Mel          54     356040   
8   0c7uc4pA1hpla0xbbEF5W5  Companhia do Calypso          47      87252   
9   1tGLEVy901arulIovZtdLc       Luan Estilizado          63     443186   
10  2iFbk7tcTw8ppznLl3cCkf        Forró do Muido          39      98949   
11  6cAXmjYOHC2R7jLrs96KTZ     Mastruz Com Leite          55     340384   
12  4rTf6JXDpcz7dsRZVPRPj

In [14]:
albums = get_artist_albums(artist_id)
authorial_albums = get_authorial_albums(albums)
print(authorial_albums)

{'74U6cBk4E34Z0MjZhRndvt': 'Calcinha Preta: Edição Especial', '1IAZzWKVMqmi781vpzX4D7': 'Balada Prime', '6fhNs80ovNuhyebRNTfj3W': 'Filmes e Histórias, Vol. 28', '0IEtoDZ9i6BMNqQj3MJzg8': 'Calcinha Preta Premium', '68trmccYCO3W5f38lctCMt': 'Eu Amo Você, Vol. 26', '6qEoBpIMERNsaufMq5VcOe': 'Meu Primeiro Namorado, Vol. 25', '5asIw5Tpu3BgXM2N5LNzWV': '15 Anos: Ao Vivo em Maceió, Vol. 24', '01LZ7gSrxSSa28j53pfa8Q': 'Virei Seu Fã, Vol. 23 (Ao Vivo)', '0ro6MVNF8IKNzMzf3VMkdJ': 'Sou Assim, Não Vou Mudar, Vol. 22', '5ZVTOnhnPcGcwsUfQediaL': 'Eu Amo Demais, Vol. 21 (Ao Vivo)', '2bTiQmCVrypuSkM4MYgzoE': 'Você Não Vale Nada, Vol. 20 (Ao Vivo)', '7kai3hgop4FikRG6ffly76': 'Vencedor, Vol. 19', '64yJROoReh7lWj1QZ7Mqc0': 'Ao Vivo no Recife, Vol. 18', '7H2JYS4S6CQyw2dbD5XHeI': 'Fica Comigo, Paulinha, Vol. 17', '2mP0EjtoHTgSjzAfpzx88R': 'Como Vou Deixar Você?, Vol. 16', '7737mniyL3zpqohpUQ1FM5': 'Pensão Alimentícia, Vol. 15', '6z8nG4kqSo7LYcWYef66Jr': 'As 20 +', '1ciAzTvbD9D3GDOgQw8VGV': 'Dois Amores, Du

In [15]:
melhoralbum = get_album_tracks('5r5rYYbGdFZXincPbvekGy')
print(melhoralbum)

{'2TWFcqzZ6KEUAYie7wEF1f': {'album': 'Calcinha Preta, Vol. 13 (Ao Vivo)', 'album_type': 'album', 'track_id': '2TWFcqzZ6KEUAYie7wEF1f', 'track_number': 1, 'name': 'Um Degrau na Escada (Ao Vivo)', 'popularity': 44, 'duration_ms': 215520, 'release_date': '2005', 'artists': ['Calcinha Preta']}, '6nXGLICKEQJv5BMZxgIrjH': {'album': 'Calcinha Preta, Vol. 13 (Ao Vivo)', 'album_type': 'album', 'track_id': '6nXGLICKEQJv5BMZxgIrjH', 'track_number': 2, 'name': 'Adão e Eva (Ao Vivo)', 'popularity': 30, 'duration_ms': 246040, 'release_date': '2005', 'artists': ['Calcinha Preta']}, '1mPcdV4w5LDxGR14pQQDke': {'album': 'Calcinha Preta, Vol. 13 (Ao Vivo)', 'album_type': 'album', 'track_id': '1mPcdV4w5LDxGR14pQQDke', 'track_number': 3, 'name': 'Hoje á Noite "Alone" (Ao Vivo)', 'popularity': 51, 'duration_ms': 212960, 'release_date': '2005', 'artists': ['Calcinha Preta']}, '0h2XejoLfJrbmy8UpfYev9': {'album': 'Calcinha Preta, Vol. 13 (Ao Vivo)', 'album_type': 'album', 'track_id': '0h2XejoLfJrbmy8UpfYev9', 

In [16]:
all_albums = get_all_albums_tracks(authorial_albums)

Adicionando:  74U6cBk4E34Z0MjZhRndvt Calcinha Preta: Edição Especial
Adicionando:  1IAZzWKVMqmi781vpzX4D7 Balada Prime
Adicionando:  6fhNs80ovNuhyebRNTfj3W Filmes e Histórias, Vol. 28
Adicionando:  0IEtoDZ9i6BMNqQj3MJzg8 Calcinha Preta Premium
Adicionando:  68trmccYCO3W5f38lctCMt Eu Amo Você, Vol. 26
Adicionando:  6qEoBpIMERNsaufMq5VcOe Meu Primeiro Namorado, Vol. 25
Adicionando:  5asIw5Tpu3BgXM2N5LNzWV 15 Anos: Ao Vivo em Maceió, Vol. 24
Adicionando:  01LZ7gSrxSSa28j53pfa8Q Virei Seu Fã, Vol. 23 (Ao Vivo)
Adicionando:  0ro6MVNF8IKNzMzf3VMkdJ Sou Assim, Não Vou Mudar, Vol. 22
Adicionando:  5ZVTOnhnPcGcwsUfQediaL Eu Amo Demais, Vol. 21 (Ao Vivo)
Adicionando:  2bTiQmCVrypuSkM4MYgzoE Você Não Vale Nada, Vol. 20 (Ao Vivo)
Adicionando:  7kai3hgop4FikRG6ffly76 Vencedor, Vol. 19
Adicionando:  64yJROoReh7lWj1QZ7Mqc0 Ao Vivo no Recife, Vol. 18
Adicionando:  7H2JYS4S6CQyw2dbD5XHeI Fica Comigo, Paulinha, Vol. 17
Adicionando:  2mP0EjtoHTgSjzAfpzx88R Como Vou Deixar Você?, Vol. 16
Adicionando:  773

In [17]:
convert_to_csv(all_albums, 'calcinhapreta')

,album,album_type,track_id,track_number,name,popularity,duration_ms,release_date,artists
0,Calcinha Preta: Edição Especial,album,28P5hnigrW5mQ2aZ439cEn,1,Quer Conhecer? Deixe!,19,228827,2018-08-03,[Calcinha Preta]
1,Calcinha Preta: Edição Especial,album,53tDhIui60U8H4CskOxOdM,2,Não Me Deixe Agora,21,293162,2018-08-03,[Calcinha Preta]
2,Calcinha Preta: Edição Especial,album,0OoMcqlselvg5wIhek1irs,3,Agora Estou Sofrendo,22,251580,2018-08-03,[Calcinha Preta]
3,Calcinha Preta: Edição Especial,album,7AARwThoxsUZ5iiIfSdkwK,4,Sem Explicação (I Will Be),20,221083,2018-08-03,[Calcinha Preta]
4,Calcinha Preta: Edição Especial,album,3lpaj83oEG9Sfx0vf4ckHv,5,Perdoar,18,186984,2018-08-03,[Calcinha Preta]


## Visualização de Dados

In [18]:
import altair as alt

data = pd.read_csv('../data/calcinhapreta.csv', encoding = 'utf-8')

In [19]:
data.head(5)

,Unnamed: 0,album,album_type,track_id,track_number,name,popularity,duration_ms,release_date,artists
0,0,Calcinha Preta: Edição Especial,album,28P5hnigrW5mQ2aZ439cEn,1,Quer Conhecer? Deixe!,19,228827,2018-08-03,['Calcinha Preta']
1,1,Calcinha Preta: Edição Especial,album,53tDhIui60U8H4CskOxOdM,2,Não Me Deixe Agora,21,293162,2018-08-03,['Calcinha Preta']
2,2,Calcinha Preta: Edição Especial,album,0OoMcqlselvg5wIhek1irs,3,Agora Estou Sofrendo,22,251580,2018-08-03,['Calcinha Preta']
3,3,Calcinha Preta: Edição Especial,album,7AARwThoxsUZ5iiIfSdkwK,4,Sem Explicação (I Will Be),20,221083,2018-08-03,['Calcinha Preta']
4,4,Calcinha Preta: Edição Especial,album,3lpaj83oEG9Sfx0vf4ckHv,5,Perdoar,18,186984,2018-08-03,['Calcinha Preta']


In [20]:
range_ = ['#222437', '#B183AE']

In [48]:
data_top20 = data.sort_values('popularity', ascending = False).head(10)
alt.Chart(data_top20).mark_circle(color = '#222437', opacity = 1).encode(
    y = alt.Y('name', axis = alt.Axis(title='Nome'), sort = None),
    x = alt.X('popularity', axis = alt.Axis(title = "Popularidade")),
    color = alt.Color('album_type', legend=alt.Legend(title="Tipo do álbum"), scale=alt.Scale(range = range_)),
    tooltip = ['name', 'popularity', 'album', 'album_type']
).configure_axis(
    labelFontSize=14
).properties(
    width=500,
    height=300,
)

alt.Chart(...)

In [46]:
data_album_most_popular = data[data['album_type'] == 'album']

bars = alt.Chart(data_album_most_popular).mark_bar(color = '#222437').encode(
    alt.X('median(popularity):Q', axis = alt.Axis(title='Mediana da Popularidade')),
    alt.Y('album:O',sort = '-x',axis = alt.Axis(title='Álbum'))
)
text = alt.Chart(data_album_most_popular).mark_text(dx=-11, dy=3, color='white').encode(
    alt.X('median(popularity):Q', axis = alt.Axis(title='Mediana da Popularidade')),
    alt.Y('album:O',sort = '-x',axis = alt.Axis(title='Álbum')),
    alt.Text('median(popularity):Q')
)
bars + text

alt.LayerChart(...)

In [49]:
data_single_most_popular = data[data['album_type'] == 'single']

bars = alt.Chart(data_single_most_popular).mark_bar(color = '#222437').encode(
    alt.X('median(popularity):Q', axis = alt.Axis(title='Mediana da Popularidade')),
    alt.Y('album:O',sort = '-x',axis = alt.Axis(title='Single'))
).properties(
    width=800
)
text = alt.Chart(data_single_most_popular).mark_text(dx=-11, dy=3, color='white').encode(
    alt.X('median(popularity):Q', axis = alt.Axis(title='Mediana da Popularidade')),
    alt.Y('album:O',sort = '-x',axis = alt.Axis(title='Single')),
    alt.Text('median(popularity):Q')
).properties(
    width=800
)
bars + text

alt.LayerChart(...)

In [24]:
N = 10
data_shortest_songs = data.sort_values(by='duration_ms', ascending=True).head(N)
data_longest_songs = data.sort_values(by='duration_ms', ascending=False).head(N)
# ver albuns repetidos

In [57]:
chart = alt.Chart(data_shortest_songs).mark_bar(color = '#222437').encode(
    alt.Y('name:N', sort='-x', axis = alt.Axis(title='Música')),
    alt.X('duration_ms:Q', axis = alt.Axis(title='Duração(ms)')),
    tooltip = ['name', 'album']
).properties(
    width=800
)
text = alt.Chart(data_shortest_songs).mark_text(dx=-25, dy=3, color='white').encode(
    alt.Y('name:N', sort='-x'),
    alt.X('duration_ms:Q'),
    alt.Text('duration_ms:Q')
).properties(
    width=800
)
chart + text

alt.LayerChart(...)

In [26]:
chart = alt.Chart(data_longest_songs).mark_bar(color = '#222437').encode(
    alt.Y('name:N', sort='-x', axis = alt.Axis(title='Música')),
    alt.X('duration_ms:Q', axis = alt.Axis(title='Duração(ms)')),
    tooltip = ['name', 'album']
)
text = alt.Chart(data_longest_songs).mark_text(dx=-25, dy=3, color='white').encode(
    alt.Y('name:N', sort='-x'),
    alt.X('duration_ms:Q'),
    alt.Text('duration_ms:Q')
)

chart + text

alt.LayerChart(...)

In [27]:
# linha do tempo de lançamentos
range_ = ['#222437', '#B183AE']
chart_circle = alt.Chart(data).mark_circle(opacity = 1).encode(
    y = alt.Y('release_date:T', axis = alt.Axis(title='Lançamento')),
    x = alt.X('album:N', sort='y', axis = alt.Axis(title='Álbum')),
    color = alt.Color('album_type', legend=alt.Legend(title="Tipo do álbum"), scale=alt.Scale(range = range_)),
    tooltip = ['album','album_type','release_date:T']
)
chart_line = alt.Chart(data).mark_line(opacity = 1).encode(
    y = alt.Y('release_date:T', axis = alt.Axis(title='Lançamento')),
    x = alt.X('album:N', sort='y', axis = alt.Axis(title='Álbum')),
    color = alt.Color('album_type', legend=alt.Legend(title="Tipo do álbum"), scale=alt.Scale(range = range_)),
    tooltip = ['album','album_type','release_date:T']
)
chart_circle + chart_line

alt.LayerChart(...)

In [58]:
#artistas relacionados
alt.Chart(related_artists).mark_circle(
    color = '#222437',
    opacity=1,
    stroke='black',
    strokeWidth=1).encode(
    alt.Y('name:N', axis = alt.Axis(title='Artistas Relacionados'), sort=None),
    alt.X('popularity:Q', axis = alt.Axis(title='Popularidade')),
    alt.Size('followers:Q')
).configure_axis(
    labelFontSize=14
).properties(
    width=500,
    height=300,
)

alt.Chart(...)